In [ ]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from source.schema.chatbot_querry import ChatbotQuery
setting=Settings()
gemini=Gemini(setting)
cohere=Cohere(setting)
bert=Bert_Extract(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

## 1.Call API System

In [ ]:


import os
import pandas as pd
from tqdm import tqdm

# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('../Eval_System/data/data_processed/final_data_eval.csv')

# Nếu chưa có cột 'answer_from_chatbot', thêm cột với giá trị None
if 'answer_from_chatbot' not in df.columns:
    df['answer_from_chatbot'] = None

# Tham số batch
batch_size = 100

def sinh_cau_tra_loi_batch(df_batch: pd.DataFrame) -> list[str]:
    """
    Sinh câu trả lời cho mỗi câu hỏi trong batch.
    Trả về danh sách answer tương ứng.
    """
    answers = []
    for question in tqdm(df_batch['question'], desc="Sinh trả lời batch"):    
        article_doc, _ = rag.get_Article_Content_Results(question)
        answers.append(article_doc)
    return answers

# Chỉ lấy các dòng chưa có câu trả lời (bị null hoặc rỗng)
df_unanswered = df[df['answer_from_chatbot'].isna() | (df['answer_from_chatbot'].str.strip() == '')]

print("==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====")
for start in range(0, len(df_unanswered), batch_size):
    end = min(start + batch_size, len(df_unanswered))
    df_batch = df_unanswered.iloc[start:end].copy()
    index_batch = df_batch.index  # Lưu lại index để cập nhật lại vào df gốc

    # Sinh câu trả lời
    df_batch['answer_from_chatbot'] = sinh_cau_tra_loi_batch(df_batch)

    # Cập nhật vào dataframe gốc theo đúng index
    df.loc[index_batch, 'answer_from_chatbot'] = df_batch['answer_from_chatbot']

    # Lưu kết quả batch (tuỳ chọn)
    batch_file = os.path.join("./data/data_processed/", f'data_final_response_{index_batch[0]}_{index_batch[-1]}.csv')
    df_batch.to_csv(batch_file, index=False)
    print(f"Lưu batch: {batch_file}")

# Ghi toàn bộ kết quả ra file chính
df.to_csv('../Eval_System/data/data_processed/final_data_eval_response.csv', index=False)
print("✅ Đã cập nhật file chính với các câu trả lời mới!")


==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====


Sinh trả lời batch:   0%|          | 0/100 [00:00<?, ?it/s]

Số document retrival được 17
5


Sinh trả lời batch:   1%|          | 1/100 [00:26<43:00, 26.06s/it]

Số document retrival được 27
5


...

Sinh trả lời batch:  53%|█████▎    | 53/100 [14:40<10:11, 13.01s/it]

Số document retrival được 16
An error occurred: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
0


Sinh trả lời batch:  54%|█████▍    | 54/100 [14:54<10:10, 13.27s/it]

Số document retrival được 16
An error occurred: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
0


Sinh trả lời batch:  55%|█████▌    | 55/100 [15:07<10:02, 13.39s/it]

### 2.Eval System

> Configure

In [ ]:
setting=Settings()

In [8]:
genai.configure(api_key=gemini.key_manager.get_next_key())

> Get embedding

In [5]:
def get_gemini_embedding(text: str,
                         model_name: str = "models/text-embedding-004") -> list[float]:
    res = genai.embed_content(
        model=model_name,
        content=text
    )
    return res["embedding"]

In [ ]:
df = pd.read_csv('./data/data_processed/data_final_response_100_199.csv')
# Lấy embedding cho Answer và answer_from_chatbot
df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_chatbot_embedding'] = df['answer_from_chatbot'].apply(get_gemini_embedding)

# Tính cosine similarity giữa Answer và answer_from_chatbot
df['cosine_similarity'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']],
        [row['answer_from_chatbot_embedding']]
    )[0][0],
    axis=1
)
# Tính trung bình của cosine similarity
average_cosine_similarity = df['cosine_similarity'].mean()

print(f"Trung bình cosine similarity giữa Answer và answer_from_chatbot: {average_cosine_similarity:.4f}")